<a href="https://colab.research.google.com/github/Nagi2003/BasketBall_player_Salary_Prediction/blob/main/Kera_tuner_hyperoarameter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('/content/diabetes.csv')

In [ ]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [ ]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [ ]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler()
X=scaler.fit_transform(X)


In [ ]:
X.shape

(768, 8)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train

array([[-0.54791859, -1.15469351, -3.57259724, ..., -4.06047387,
        -0.50700636, -1.04154944],
       [ 1.53084665, -0.27837344,  0.66661825, ..., -0.48135115,
         2.44666971,  1.4259954 ],
       [-0.84488505,  0.56664949, -1.19450074, ..., -0.41789153,
         0.55003518, -0.95646168],
       ...,
       [ 1.82781311, -0.62264204,  0.87340925, ...,  1.72704372,
         2.00573238,  0.40494237],
       [-1.14185152,  0.62924378, -3.57259724, ...,  1.32090213,
        -0.8059981 , -0.36084741],
       [-1.14185152,  0.12848945,  1.39038675, ..., -1.20479085,
        -0.63385134, -1.04154944]])

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
pip install -U keras-tuner

In [ ]:
import kerastuner as kt

In [ ]:
def build_model(hp):
  model=Sequential()
  counter=0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(Dense(
          hp.Int('units'+str(i),min_value=1,max_value=128,step=8),
          activation=hp.Choice('activation'+str(i),values=['relu','sigmoid','tanh']),input_dim=8
      ))
      model.add(Dropout(hp.Choice('Dropout',values=['0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9'])))
    else:
      model.add(Dense(
          hp.Int('units'+str(i),min_value=1,max_value=128,step=8),
          activation=hp.Choice('activation'+str(i),values=['relu','sigmoid','tanh'])
      ))
      model.add(Dropout(hp.Choice('Dropout',values=['0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9'])))
    counter+=1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
  return model




In [ ]:
tuner=kt.RandomSearch(build_model,
                      objective='accuracy',
                      max_trials=3,
                      )

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 89,
 'activation0': 'sigmoid',
 'optimizer': 'rmsprop',
 'units1': 57,
 'activation1': 'relu',
 'units2': 41,
 'activation2': 'relu',
 'units3': 65,
 'activation3': 'relu',
 'units4': 1,
 'activation4': 'sigmoid',
 'units5': 1,
 'activation5': 'tanh',
 'units6': 89,
 'activation6': 'relu',
 'units7': 33,
 'activation7': 'relu',
 'units8': 25,
 'activation8': 'sigmoid',
 'units9': 73,
 'activation9': 'sigmoid'}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'dropout' (type Dropout).
    
    `rate` must be a scalar or scalar tensor. Received: rate=0.1
    
    Call 